In [1]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
import time

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.6.0
sys.version_info(major=3, minor=7, micro=9, releaselevel='final', serial=0)
matplotlib 3.4.1
numpy 1.20.2
pandas 1.2.3
sklearn 0.24.1
tensorflow 2.6.0
keras.api._v2.keras 2.6.0


In [4]:
train_dir = "./cifar-10/train"
test_dir = "./cifar-10/test"
valid_dir = './cifar-10/validation'
print(os.path.exists(train_dir))
print(os.path.exists(test_dir))
print(os.path.exists(valid_dir))

print(os.listdir(train_dir))
print(os.listdir(test_dir))
print(os.listdir(valid_dir))

True
True
True
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [5]:
height = 32
width = 32
channels = 3
batch_size = 32
num_classes = 10

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest',
)
train_generator = train_datagen.flow_from_directory('./cifar-10/train',
                                                   target_size = (height, width),
                                                   batch_size = batch_size,
                                                   seed = 7,
                                                   shuffle = True,
                                                   class_mode = "sparse")
valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                    target_size = (height, width),
                                                    batch_size = batch_size,
                                                    seed = 7,
                                                    shuffle = False,
                                                    class_mode = "sparse")
train_num = train_generator.samples
valid_num = valid_generator.samples
print(train_num, valid_num)

Found 45000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.
45000 5000


In [6]:
for i in range(2):
    x, y = train_generator.next()
    print(x.shape, y.shape)
    print(y)

(32, 32, 32, 3) (32,)
[5. 1. 2. 4. 7. 6. 9. 7. 6. 9. 6. 2. 2. 5. 9. 0. 4. 0. 5. 1. 5. 8. 1. 4.
 4. 7. 5. 3. 8. 5. 5. 0.]
(32, 32, 32, 3) (32,)
[7. 5. 0. 7. 3. 1. 4. 1. 8. 3. 8. 2. 5. 1. 2. 8. 1. 6. 3. 3. 4. 8. 8. 6.
 6. 5. 7. 3. 8. 3. 6. 5.]


In [7]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same',
                        activation='relu', 
                        input_shape=[width, height, channels]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=256, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(filters=512, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=512, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax'),
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 256)       1

In [8]:
epochs = 20
history = model.fit_generator(train_generator,
                              steps_per_epoch = train_num // batch_size,
                              epochs = epochs,
                              validation_data = valid_generator,
                              validation_steps = valid_num // batch_size)

c:\users\lizhi\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
1406/1406 [==============================] - 821s 584ms/step - loss: 1.9760 - accuracy: 0.3543 - val_loss: 1.8646 - val_accuracy: 0.3850
Epoch 2/20
1406/1406 [==============================] - 842s 599ms/step - loss: 1.4186 - accuracy: 0.4890 - val_loss: 1.4173 - val_accuracy: 0.5367
Epoch 3/20
1406/1406 [==============================] - 843s 600ms/step - loss: 1.2169 - accuracy: 0.5705 - val_loss: 1.1733 - val_accuracy: 0.6202
Epoch 4/20
1406/1406 [==============================] - 842s 599ms/step - loss: 1.0594 - accuracy: 0.6319 - val_loss: 0.9852 - val_accuracy: 0.6789
Epoch 5/20
1406/1406 [==============================] - 856s 609ms/step - loss: 0.9337 - accuracy: 0.6766 - val_loss: 0.9596 - val_accuracy: 0.7025
Epoch 6/20
1406/1406 [==============================] - 869s 618ms/step - loss: 0.8505 - accuracy: 0.7087 - val_loss: 0.8648 - val_accuracy: 0.7171
Epoch 7/20
1406/1406 [==============================] - 898s 639ms/step - loss: 0.7734 - accuracy: 0.7368 - val_

In [9]:
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [11]:
test_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (height, width),
    batch_size = batch_size,
    seed = 7,
    shuffle = False,
    class_mode = "sparse")
test_num = test_generator.samples
print(test_num)

Found 10000 images belonging to 10 classes.
10000


In [ ]:
# print(history.history)
def plot_learning_curves(history, label, epcohs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()
    
plot_learning_curves(history, 'accuracy', epochs, 0, 1)
plot_learning_curves(history, 'loss', epochs, 0, 2)